# Station data processing

In [29]:
import Glob
import Dates
import PyCall
import FTPClient

import CSV
import DataFrames

**Download some station data**

In [30]:
ftp = FTPClient.FTP("ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/")
ftp_filename = "by_year/2019.csv.gz";
ftp_metaname = "ghcnd-stations.txt"

# download to a local path
filename = "/glade/u/home/ksha/WORKSPACE/JL_Collection/data/2019.csv.gz";
metaname = "/glade/u/home/ksha/WORKSPACE/JL_Collection/data/ghcnd-stations.txt";
FTPClient.download(ftp, ftp_filename, filename);
FTPClient.download(ftp, ftp_metaname, metaname);
FTPClient.close(ftp)

In [5]:
# call the system cmd to unpack *.gz
run(`gunzip $filename`)

Process(`gunzip /glade/u/home/ksha/WORKSPACE/JL_Collection/data/2019.csv.gz`, ProcessExited(0))

In [4]:
filename = filename[1:end-3]

"/glade/u/home/ksha/WORKSPACE/JL_Collection/data/2019.csv"

**Import csv file as DataFrame**

In [33]:
df = CSV.read(filename, header=false);

# assigning column names based on the online documentation
# ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt

colnames = ["stn ID", "date", "variable", "value", "measurement_flag", "quality_flag", "network_flag", "obs_flag"];
DataFrames.rename!(df, colnames);

In [64]:
df[1:4, :] # first 4 rows

,stn ID,date,variable,value,measurement_flag,quality_flag,network_flag
,String,Int64,String,Int64,String⍰,String⍰,String
1,US1FLSL0019,20190101,PRCP,0,missing,missing,N
2,US1FLSL0019,20190101,SNOW,0,missing,missing,N
3,US1NVNY0012,20190101,PRCP,0,missing,missing,N
4,US1NVNY0012,20190101,SNOW,0,missing,missing,N


**DataFrame processing**

In [81]:
# subsetting daily mean 2-m temperature
df_t2m = filter(df_temp -> any(occursin.(["TAVG"], df_temp.variable)), df);

# drop network_flag
DataFrames.select!(df_t2m, DataFrames.Not(:network_flag));

# selecting good flags
df_t2m = df_t2m[ismissing.(df_t2m[!, :measurement_flag]), :];
df_t2m = df_t2m[ismissing.(df_t2m[!, :quality_flag]), :];
df_t2m = df_t2m[ismissing.(df_t2m[!, :obs_flag]), :];

# drop useless columns
DataFrames.select!(df_t2m, DataFrames.Not(:measurement_flag));
DataFrames.select!(df_t2m, DataFrames.Not(:quality_flag));
DataFrames.select!(df_t2m, DataFrames.Not(:obs_flag));
DataFrames.select!(df_t2m, DataFrames.Not(:variable));

In [87]:
df_t2m[1:4, :]

,stn ID,date,value
,String,Int64,Int64
1,USS0018D08S,20190101,-56
2,CA003034485,20190101,-44
3,USS0006K44S,20190101,-117
4,USS0010E12S,20190101,-179


In [96]:
# subsetting on the date 20191231
df_t2m_191231 = filter(df_temp -> any(df_temp.date == 20191231), df_t2m);

# write into csv
exportname = "/glade/u/home/ksha/WORKSPACE/JL_Collection/data/T2m_20191231_clean.csv"
CSV.write(exportname, df_t2m_191231)

"/glade/u/home/ksha/WORKSPACE/JL_Collection/data/T2m_20191231_clean.csv"

**Import metadata (an irregular text file) as DataFrame**

In [5]:
metadata = CSV.read(metaname, header=false, delim=" ", silencewarnings=true);

In [6]:
metadata[1:10, :]

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
,String,Float64⍰,Float64⍰,Float64⍰,Float64⍰,String⍰,String⍰,String⍰
1,ACW00011604,missing,17.1167,missing,-61.7833,missing,missing,10.1
2,ACW00011647,missing,17.1333,missing,-61.7833,missing,missing,19.2
3,AE000041196,missing,25.333,missing,missing,55.5170,missing,missing
4,AEM00041194,missing,25.255,missing,missing,55.3640,missing,missing
5,AEM00041217,missing,24.433,missing,missing,54.6510,missing,missing
6,AEM00041218,missing,24.262,missing,missing,55.6090,missing,264.9
7,AF000040930,missing,35.317,missing,missing,69.0170,3366.0,missing
8,AFM00040938,missing,34.21,missing,missing,62.2280,missing,977.2
9,AFM00040948,missing,34.566,missing,missing,69.2120,1791.3,missing


In [37]:
size_df = size(metadata);

In [42]:
meta_array = Array{Float64}(undef, size_df[1], 3); # 3 = lat, lon, elev

In [63]:
# loop over rows that have float64.
# Based on the documentation, these numbers are: lat, lon, elev, and other useless words
for i = 1:size_df[1]
    temp_array = collect(skipmissing(metadata[i, :]))[2:4]
    meta_array[i, :] = [x isa Float64 ? x : parse(Float64, x) for x in temp_array]
end

In [67]:
metadata_clean = DataFrames.DataFrame(ID=metadata[:, 1], 
    lat=meta_array[:, 1], 
    lon=meta_array[:, 2], 
    elev=meta_array[:, 3]);

In [68]:
metadata_clean[1:4, :]

,ID,lat,lon,elev
,String,Float64,Float64,Float64
1,ACW00011604,17.1167,-61.7833,10.1
2,ACW00011647,17.1333,-61.7833,19.2
3,AE000041196,25.333,55.517,34.0
4,AEM00041194,25.255,55.364,10.4


In [69]:
# write into csv
exportname = "/glade/u/home/ksha/WORKSPACE/JL_Collection/data/metadata_clean.csv"
CSV.write(exportname, metadata_clean)

"/glade/u/home/ksha/WORKSPACE/JL_Collection/data/metadata_clean.csv"